# File Load

### png file load(OCR)

In [ ]:
from PIL import Image
from transformers import AutoTokenizer, AutoProcessor, AutoModelForImageTextToText

model_path = "nanonets/Nanonets-OCR-s"

model = AutoModelForImageTextToText.from_pretrained(
    model_path, 
    torch_dtype="auto", 
    device_map="auto", 
    # attn_implementation="flash_attention_2"
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_path)
processor = AutoProcessor.from_pretrained(model_path)


def ocr_page_with_nanonets_s(image_path, model, processor, max_new_tokens=4096):
    prompt = """Extract the text from the above document as if you were reading it naturally. Return the tables in html format. Return the equations in LaTeX representation. If there is an image in the document and image caption is not present, add a small description of the image inside the <img></img> tag; otherwise, add the image caption inside <img></img>. Watermarks should be wrapped in brackets. Ex: <watermark>OFFICIAL COPY</watermark>. Page numbers should be wrapped in brackets. Ex: <page_number>14</page_number> or <page_number>9/22</page_number>. Prefer using ☐ and ☑ for check boxes."""
    image = Image.open(image_path)
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": [
            {"type": "image", "image": f"file://{image_path}"},
            {"type": "text", "text": prompt},
        ]},
    ]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[image], padding=True, return_tensors="pt")
    inputs = inputs.to(model.device)
    
    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, output_ids)]
    
    output_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return output_text[0]


In [5]:
import os
import glob
import re
from PIL import Image

def extract_number(filename):
    # 파일 이름에 포함된 숫자 부분만 찾아서 반환
    numbers = re.findall(r'(\d+)', filename)
    return int(numbers[-1]) if numbers else -1 # 숫자가 여러 개 있을 경우 뒤의 숫자 사용

input_dir = "./data/중소기업은행_금융시장 동향_20231231"  # 원본 이미지 폴더
output_dir = "./data/중소기업은행_금융시장 동향_20231231/resized_images"  # 변환 이미지 저장 폴더
os.makedirs(output_dir, exist_ok=True)

# 모든 png 파일 탐색
images_files = glob.glob(os.path.join(input_dir, "*.png"))

# 자연수 기준 정렬 (1, 11, 12, 2, 3 순서 방지)
image_files.sort(key=lambda x: extract_number(os.path.basename(x)))

for img_path in image_files:
    try:
        # 이미지 열기 및 resize
        image = Image.open(img_path)
        resized_image = image.resize((image.width // 2, image.height // 2))

        # 저장 파일명 : 원본과 동일하게, 저장 디렉토리에 저장
        base = os.path.splitext(os.path.basename(img_path))[0]
        save_path = os.path.join(output_dir, f"{base}.png")
        resized_image.save(save_path)
        print(f"Saved resized image: {save_path}")
    except Exception as e:
        print(f"파일 처리 중 오류 발생: {img_path} ({e})")

Saved resized image: ./data/예금보험공사_금융상품 동향_20240401/resized_images/금융상품_동향_브리프(24년_4월호)_1.png
Saved resized image: ./data/예금보험공사_금융상품 동향_20240401/resized_images/금융상품_동향_브리프(24년_4월호)_2.png
Saved resized image: ./data/예금보험공사_금융상품 동향_20240401/resized_images/금융상품_동향_브리프(24년_4월호)_3.png
Saved resized image: ./data/예금보험공사_금융상품 동향_20240401/resized_images/금융상품_동향_브리프(24년_4월호)_4.png
Saved resized image: ./data/예금보험공사_금융상품 동향_20240401/resized_images/금융상품_동향_브리프(24년_4월호)_5.png
Saved resized image: ./data/예금보험공사_금융상품 동향_20240401/resized_images/금융상품_동향_브리프(24년_4월호)_6.png
Saved resized image: ./data/예금보험공사_금융상품 동향_20240401/resized_images/금융상품_동향_브리프(24년_4월호)_7.png
Saved resized image: ./data/예금보험공사_금융상품 동향_20240401/resized_images/금융상품_동향_브리프(24년_4월호)_8.png
Saved resized image: ./data/예금보험공사_금융상품 동향_20240401/resized_images/금융상품_동향_ᄇ

In [ ]:
input_dir = "./data/예금보험공사_금융상품 동향_20240401/resized_images"    # OCR 대상 이미지 폴더
output_txt = "./data/예금보험공사_금융상품 동향_20240401/금융동향분석_전체.txt"

# 모든 png 파일 탐색
image_files = glob.glob(os.path.join(input_dir, "*.png"))

# 자연수 기준 정렬
image_files.sort(key=lambda x: extract_number(os.path.basename(x)))

with open(output_txt, "w", encoding="utf-8") as out_f:
    for idx, img_path in enumerate(image_files, 1):
        print(f"OCR 진행 중: {img_path}")
        result = ocr_page_with_nanonets_s(img_path, model, processor, max_new_tokens=2048)
        out_f.write(f"\n\n--- Page {idx} ({os.path.basename(img_path)}) ---\n\n")
        out_f.write(result)
        print(f"페이지 {idx} 결과 누적 완료")

print(f"[완료] 전체 보고서가 '{output_txt}'에 저장되었습니다.")

### hwp file load

In [3]:
from langchain_teddynote.document_loaders import HWPLoader

# HWP Loader 객체 생성
loader = HWPLoader("./data/중소기업은행_금융시장 동향_20231231/자금운용부모닝커피와마켓(20230102)_’23년 금융시장 리스크 점검.hwpx")

# 문서 로드
docs = loader.load()

In [4]:
docs[0].page_content

'   년   월   일  ’23년 금융시장 리스크 점검 지난해 전례 없는 고강도의 기준금리 인상으로 금융시장은 큰 변동성1)을 보였는데, 새해를 시작하며 ’23년에 주목해 볼 주요 리스크 요인을 짚어보고자 함.(주요 리스크)① 고금리 부작용 - 물가가 확실하게 안정되기 전까지 고금리 정책 유지 전망2)  - 높은 가계부채, 한계기업 부담가중, 자산가격(부동산 등) 하락, 유동성경색 등 우려② 지정학적 리스크 - 러-우 전쟁 장기화에 따른 안보 및 에너지위기, 러시아 돌발위험 잠복 - 중국-대만, 사우디-이란 분쟁가능성, ’24년 대선을 앞둔 미국 대응에 따른 불확실성③ 분절화(탈세계화) - - 패권경쟁 심화( : 리쇼어링, IT ․ 금융 중제재 vs  : 기술자립으로 대응) - 자국 우선주의, 블록화 등 글로벌 교역환경 변화는 세계경제에 부정적3)(시사점) 상기의 리스크 요인들이 글로벌 경제회복을 지연시킬 우려도 있으나 이를 선제적으로 대비하고 위기를 기회로 삼는다면, 우리나라가 경기 연착륙에 성공하는 희망의 한 해가 될 수 있을 것으로 기대.     1) ’22년 금융시장 변동 ( 기준금리 +225bp 인상하여 3.25%,  기준금리 +425bp 인상하여 4.50%)  (단위 : pt, 원, %)’21년연중 최저 / 최고’22년전년대비KOSPI2,977.652,134.77 / 3,010.772,236.40△741.25 (△24.9%)S&P5004,766.183,491.58 / 4,818.623,839.50△926.68 (△19.4%)달러/원 환율1,188.81,185.6 / 1,444.21,264.5+75.7  (+6.4%)국고채3년 금리1.7981.795 / 4.5553.722+192bp     2) ’23년 기준금리 상단 전망치 :  3.50~3.75%,  5.00~5.25%     3) 분절화 시뮬레이션 결과, 무역량 감소와 자원의 비효율적 배분으로 인해 전세계 실질소득 △5% 감소 예상(출처 : IMF)   숫자로 보는 뉴스      44.930일 

### txt file load

In [13]:
from langchain_community.document_loaders import TextLoader

# 텍스트 로더 생성
loader = TextLoader("data/블로그_텍스트/blog_text1.txt")

# 문서 로드
docs = loader.load()
print(f"문서의 수: {len(docs)}\n")
print("[메타데이터]\n")
print(docs[0].metadata)
print("\n========= [앞부분] 미리보기 =========\n")
print(docs[0].page_content[:500])

문서의 수: 1

[메타데이터]

{'source': 'data/블로그_텍스트/blog_text1.txt'}

========= [앞부분] 미리보기 =========

“월급만으로 미래 준비하고 계신가요?”
요즘 같은 고물가 시대에 단순한 저축만으로는 안정적인 미래를 기대하기 어렵습니다.
하지만 바쁜 직장 생활 속에서도 실천할 수 있는 재테크 루틴은 분명히 존재합니다.
이번 글에서는 직장인이 지금 당장 실천 가능한 현실적 재테크 전략 6가지를 소개합니다.

[1] 통장 쪼개기: 월급 통장을 적극적으로 관리하라
많은 직장인들이 월급이 들어오는 통장을 하나로만 관리합니다.
하지만 목적에 따라 통장을 구분하면 불필요한 소비를 줄이고 자산 흐름을 파악하기 쉬워집니다.

통장 종류
급여 통장 : 월급 수령 전용 (자동이체 최소화)
고정지출 통장 : 카드, 공과금, 보험 등 고정비용 이체용
저축 통장 : 매달 자동 이체로 저축
투자 통장 : 소액 투자, ETF, 펀드 등 자산 운용

[2] 월 10만 원부터 시작하는 소액 투자
많은 직장인들이 투자를 시작하기를 두려워합니다. 그러나 투자 경험이 없더라도 소액부터 시작하는 투자는 필수입니다.
추천 투자 방법
적


In [14]:
loader = TextLoader("./data/예금보험공사_금융상품 동향_20240401/resized_images/금융동향분석_전체.txt")
docs_2 = loader.load()
print(docs_2[0].page_content[:500])

## 2024. 4.


I. 금융상품 동향
주목된 상품들의 주요 내용 및 유의사항
보유가있으면 건강한 철저도 소개 및 유의사항
금융상품 동향, 시장변화에 따른 경쟁기준을 개선합니다.
금융상품 동향은 정당한 금융상품 기업산출을 위장한 자료 사이트에 의해

II. 금융상품 동향
- 산업계를 통합
- 금융상품 동향 동향

1. 금융상품 등장

* 주택담보대출의 주요 내용 및 유의사항

☑ 주택담보대출(이하 주담대)이란 금융회사에 주택(아파트, 주거용 오피스텔 등)을 담보로 제공하고 이를 빌리는 대출 상품임.
☑ 주담대의 대출금은 임대인부담비(LOI), 충당금(LOI), 충당금이외의 충당금이 포함될 수 있음.
☑ 주담대는 주택의 가치를 보증하는 대출로, 주택의 가치가 변동할 경우, 금융기관이 주택가치를 평가하여 특정 시점에서 변동을 반영하는 충당금이 필요함.
☑ 주택담보대출은 대출자와 담보자 간의 거래로 이루어져 있으며, 이는 주택가치 변동에 따라 결정되며, 변동성으로 인해 대출자에게는 


### 디렉토리 자동 load

In [15]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"

text_loader_kwargs = {"autodetect_encoding": True}

loader = DirectoryLoader(
    path,
    glob="**/*.txt",
    loader_cls=TextLoader,
    silent_errors=True,
    loader_kwargs=text_loader_kwargs,
)
docs = loader.load()

In [16]:
print(docs[0].page_content[:500])

​
​
안녕하세요, 재테크하는제인입니다.
최근 미국의 지니어스 법안이 통과되면서 가상화폐 시장이 활기를 띠고 있는 가운데 대표적인 안전자산인 금 값이 잠시 주춤했었지요.
​
원래 금은 안전자산이기 때문에 장기 보유 하고 있는 투자자들이 많고 글로벌 위기가 닥쳤을 때는 시세가 급상승 하는 경우도 많았습니다.
​
그렇다면 지금은 완전한 글로벌 리스크 해소가 되었기 때문에 시세가 떨어진걸까요? 아닙니다.
​
당연히 지금 현재의 상황은 예전보다 전쟁이 심화되거나 하진 않지만 분명한 리스크가 존재하는데도 3개월째 계속 유지중인 이유가 있겠지요.
​
오늘은 그래서 금투자방법 중에서도 금현물에 투자하는 국내 금 ETF 4가지를 비교해보고 각각의 전략도 고려 해보겠습니다.
​
1. 금시세
​
​
현재 금 1돈 3.75G의 가격은 약 56만4천원입니다. 한 때 63만원을 넘겼던 것을 생각하면 지금 한풀 꺾였다고 봐야해요.
​
2월에 g당 168,500원으로 고점을 찍은 이후 약 10% 정도 빠졌다고


In [17]:
print(docs[72].page_content[:500])

사회초년생 재테크
11. 금융상품 광고 바로 읽기
글·사진 유니콘
안녕하세요, 유니콘입니다!
적금, 예금을 들려고
은행 금융상품을 살펴보면
“연 7% 적금 금리!”, “무조건 원금 보장!”,
“수익률 100% 경험하세요!”
이런 문구들, 다들 한 번쯤 봐보셨죠?
​
사회초년생 시절의 저도 혹해서
상품에 가입한 적이 있었어요.
나중에 자세히 보니
반짝 혜택과
​
숨은 조건
들이 참 많더라구요.
​
오늘은 사회초년생이 금융상품 광고에서
꼭 주의해야 할 포인트와
광고 문구 속 진짜 의미
파악하는 방법을 정확하게 알려드릴게요.
​
✅ 광고 문구에 숨겨진 함정, 이렇게 읽으세요
​
① “연 최대 7% 금리!”
👉
기본금리 + 우대조건 = 최대금리
​
대부분의 고금리 적금은
기본 금리는 2~3%
,
나머지는 우대금리로 구성돼 있어요.
​
우대금리는
급여이체, 자동이체,
카드 실적, 앱 로그인, 신규 가입자
등
조건이 빡빡한 경우가 많아요.
​
🔍 꼭 확인하세요
✔️기본금리는 몇 %인지
✔️우대


# text split

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len, # 문자열 길이를 계산하는 함수를 지정
    is_separator_regex=False # 구분자로 정규식을 사용할지 여부를 설정
)

In [19]:
chunks = text_splitter.split_documents(docs)

In [20]:
chunks

[Document(metadata={'source': 'data/블로그_텍스트/blog_text60.txt'}, page_content='\u200b\n\u200b\n안녕하세요, 재테크하는제인입니다.\n최근 미국의 지니어스 법안이 통과되면서 가상화폐 시장이 활기를 띠고 있는 가운데 대표적인 안전자산인 금 값이 잠시 주춤했었지요.\n\u200b\n원래 금은 안전자산이기 때문에 장기 보유 하고 있는 투자자들이 많고 글로벌 위기가 닥쳤을 때는 시세가 급상승 하는 경우도 많았습니다.\n\u200b\n그렇다면 지금은 완전한 글로벌 리스크 해소가 되었기 때문에 시세가 떨어진걸까요? 아닙니다.\n\u200b\n당연히 지금 현재의 상황은 예전보다 전쟁이 심화되거나 하진 않지만 분명한 리스크가 존재하는데도 3개월째 계속 유지중인 이유가 있겠지요.\n\u200b\n오늘은 그래서 금투자방법 중에서도 금현물에 투자하는 국내 금 ETF 4가지를 비교해보고 각각의 전략도 고려 해보겠습니다.\n\u200b\n1. 금시세\n\u200b\n\u200b\n현재 금 1돈 3.75G의 가격은 약 56만4천원입니다. 한 때 63만원을 넘겼던 것을 생각하면 지금 한풀 꺾였다고 봐야해요.\n\u200b\n2월에 g당 168,500원으로 고점을 찍은 이후 약 10% 정도 빠졌다고 보면 됩니다. 그런데 이미 가격이 많이 올랐기 때문에 10%라고 하더라도 꽤 시세가 빠진 것을 확인 할 수 있습니다.\n\u200b\n그런데 그 이후로 3개월 정도 횡보하고 있는 수순이라 과연 횡보가 길어질지 아니면 상승할지는 지켜봐야 할 것 같습니다.\n\u200b\n\u200b\n국제 금시세의 경우 국내보다 변동폭이 좀 더 큰데 미국 파월 연준의장의 해임안에 대한 이야기가 나오면서 소폭 상승하더니 또 금새 가라앉았네요.\n다만, 지금 이 금 시장에서 어디의 수요가 집중되어 있는지 파악할 필요는 있다고 봅니다.\n\u200b\n2. 중국과 금의 관계\n\u200b\n계속 랠리를 이어가던 금시세가 왜 지금 주춤한 걸까요? 이는

# Embedding & Vector DB 구축

In [21]:
from dotenv import load_dotenv
import os


load_dotenv()

True

In [22]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    dimensions = 1024
)

In [23]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

# 문자열 리스트로 생성
db = FAISS.from_documents(
    chunks,
    embeddings
)

In [24]:
db.docstore.__dict__

{'_dict': {'822d4b4f-f2a5-4c37-ac5d-58780377aebb': Document(id='822d4b4f-f2a5-4c37-ac5d-58780377aebb', metadata={'source': 'data/블로그_텍스트/blog_text60.txt'}, page_content='\u200b\n\u200b\n안녕하세요, 재테크하는제인입니다.\n최근 미국의 지니어스 법안이 통과되면서 가상화폐 시장이 활기를 띠고 있는 가운데 대표적인 안전자산인 금 값이 잠시 주춤했었지요.\n\u200b\n원래 금은 안전자산이기 때문에 장기 보유 하고 있는 투자자들이 많고 글로벌 위기가 닥쳤을 때는 시세가 급상승 하는 경우도 많았습니다.\n\u200b\n그렇다면 지금은 완전한 글로벌 리스크 해소가 되었기 때문에 시세가 떨어진걸까요? 아닙니다.\n\u200b\n당연히 지금 현재의 상황은 예전보다 전쟁이 심화되거나 하진 않지만 분명한 리스크가 존재하는데도 3개월째 계속 유지중인 이유가 있겠지요.\n\u200b\n오늘은 그래서 금투자방법 중에서도 금현물에 투자하는 국내 금 ETF 4가지를 비교해보고 각각의 전략도 고려 해보겠습니다.\n\u200b\n1. 금시세\n\u200b\n\u200b\n현재 금 1돈 3.75G의 가격은 약 56만4천원입니다. 한 때 63만원을 넘겼던 것을 생각하면 지금 한풀 꺾였다고 봐야해요.\n\u200b\n2월에 g당 168,500원으로 고점을 찍은 이후 약 10% 정도 빠졌다고 보면 됩니다. 그런데 이미 가격이 많이 올랐기 때문에 10%라고 하더라도 꽤 시세가 빠진 것을 확인 할 수 있습니다.\n\u200b\n그런데 그 이후로 3개월 정도 횡보하고 있는 수순이라 과연 횡보가 길어질지 아니면 상승할지는 지켜봐야 할 것 같습니다.\n\u200b\n\u200b\n국제 금시세의 경우 국내보다 변동폭이 좀 더 큰데 미국 파월 연준의장의 해임안에 대한 이야기가 나오면서 소폭 상승하더니 또 금새 가라앉았네요.\n다만, 지금 이 금 시장에서 어디의 수

In [25]:
db.save_local("my_faiss_index")

## 유사도 검색

In [26]:
db.similarity_search("ISA 에 대하여 알려줘", k=2)
#  filter={"source": "data/finance-keywords.txt"}

[Document(id='32db2845-7193-450b-8d47-059d2c3acb64', metadata={'source': 'data/블로그_텍스트/blog_text33.txt'}, page_content='ISA 계좌 개설 방법 한도 만기 됐을 때 해지방법은?\n\u200b\nISA 계좌 써본 사람 있음?\n\u200b\n처음 가입해보려하면\n어떻게 해야할 지 헷갈리는데,\n설명은 또 왜 그렇게\n어려운 건지 원.\n\u200b\n그래서 내가 직접 나섰음.\n이놈의 ISA.\n\u200b\n어떻게 써먹어야\n우리한테 이득인지,\n아주 그냥 시원하게\n정리해주려고 함.\n\u200b\n복잡한 거 딱 질색인\n형들을 위해 핵심만\n쏙쏙 뽑아봐왔음\n\u200b\n이번엔 진짜\n제대로 알고 가자고..\n[목차]\n1. ISA 계좌란?\n2. 장점 알아보기\n3. 종류 및 조건\n4. 개설 방법\n1. ISA 계좌란?\n\u200b\n다들 알겠지만\n돈 모으고 불리는 과정에서\n세금이란 놈은\n진짜 무시 못 할 존재임.\n\u200b\n이자 좀 생겼다 싶으면\n15.4퍼센트 떼어가고,\n배당금도 마찬가지니\n아쉬울 때가 한두 번이 아님.\n\u200b\n그래서 이런 때일수록\n절세라는 두 글자가\n더 중요하게 다가오는 것 같음.\n\u200b\n이런 고민을 하는 형들에게\nISA 계좌는 꽤 괜찮은\n선택지가 될 수 있음.\n\u200b\n만능 통장이라고들 부르는데,\n잘만 활용하면 재테크에\n쏠쏠한 재미와 함께,\n\u200b\n든든한 지원군이\n되어줄 수 있을 거임.\n\u200b\n\u200b\nISA 계좌는 길게 표현해서,\n개인종합자산관리계좌의 약자임\n.\n\u200b\n하나의 계좌 안에서\n예금부터 펀드, ETF, 주식, 채권까지\n아주 다양한 금융상품을 관리\n할 수 있음!\n\u200b\n금융투자협회 자료를 슬쩍 보니\n지난달 말 기준으로\n가입자 수가 벌써\n600만 명을 넘어섰다 하니,\n\u200b\n이미 많은 형들이\n그 가치를 알아보고

## conversation history 저장하는 함수

In [27]:
from langchain_community.chat_message_histories import ChatMessageHistory
# 세션 기록을 저장할 딕셔너리
store = {}


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환

##  Q&A 구현 (RAG 활용)

In [28]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-4.1-mini"),
    retriever=db.as_retriever()
)
answer = qa.run("월급쟁이부자들이 뭐야?")
print("🧠 답변:", answer)

/var/folders/2v/7_c8xzcd0wz7syldr0lxdtk00000gn/T/ipykernel_43096/1225270396.py:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa.run("월급쟁이부자들이 뭐야?")


🧠 답변: "월급쟁이부자들"은 대한민국 직장인을 대상으로 부동산, 주식, 부업 등 재테크 강의를 제공하는 교육 플랫폼입니다. 월급만으로는 미래가 막막한 직장인들이 행복한 노후를 준비하고 내 집 마련 등 재산을 늘릴 수 있도록 돕는 곳이에요.

주요 내용은 다음과 같습니다:
- 약 60만 명의 수강생과 197만 명 이상의 유튜브 구독자를 보유한 국내 최대 규모의 재테크 교육 플랫폼입니다.
- 부동산 투자, 주식 등 재테크 관련 다양한 강의를 제공하며, 특히 직장인과 주부들을 위한 실질적이고 현실적인 투자 전략을 배울 수 있습니다.
- 현재 부동산 투자 필수 전략 등의 라이브 강의를 0원으로 체험할 수 있는 이벤트도 진행 중입니다.
- 재테크에 대해 처음 시작하는 사람도 쉽게 도전할 수 있도록 돕는 콘텐츠가 많아 많은 사람들이 긍정적인 영향을 받고 있습니다.

즉, "월급쟁이부자들"은 월급쟁이 직장인들이 현명하게 돈을 관리하고 투자할 수 있게 만들어진 종합 재테크 교육 플랫폼이라고 볼 수 있습니다.


## RAG 미활용 Q&A

In [18]:
llm = ChatOpenAI(model="gpt-4.1-mini")
question  = "월급쟁이부자들이 뭐야?"
response = llm.invoke(question)
print(response)

content='"월급쟁이 부자들"이라는 표현은 일반적으로 고정적인 월급(급여)을 받으면서도 경제적으로 부유해진 사람들을 의미합니다. 보통 \'월급쟁이\'는 직장인이나 직원들을 가리키며, 정해진 급여를 받는 사람을 말하죠. 하지만 많은 경우 월급만으로는 부자가 되기 어렵다고 알려져 있는데, "월급쟁이 부자들"은 월급 외에도 저축, 투자, 부동산, 사업 등 다양한 방법을 통해 재산을 늘리고 경제적 자유를 이룬 사람들을 뜻합니다.\n\n이 표현은 특히 재테크와 투자에 관심 있는 사람들이 자신의 경제적 상황을 개선하거나 성공 사례를 공유할 때 자주 사용됩니다.  \n\n예를 들어, 월급을 절약해 주식이나 펀드에 투자하거나 부동산에 투자하여 수익을 창출하는 방식으로 부를 축적한 사람들을 "월급쟁이 부자"라고 부를 수 있습니다.  \n\n또한 "월급쟁이 부자들"이라는 제목의 책이나 블로그, 유튜브 채널도 있는데, 이런 곳에서는 직장인들이 어떻게 재테크를 해서 부자가 될 수 있는지 실용적인 팁과 경험을 소개합니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 280, 'prompt_tokens': 17, 'total_tokens': 297, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_658b958c37', 'id': 'chatcmpl-C2f5RxisIAYPe4P2oPAtdL00nsiYl', 'service_tier

In [ ]:
answer = qa.run("지금 시간정보 알려줘")
print("🧠 답변:", answer)

🧠 답변: 죄송하지만 현재 시각 정보를 제공해 드릴 수 없습니다.
